<a href="https://colab.research.google.com/github/harshithard/ComputerVision/blob/main/HOG_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import cv2
import matplotlib.pyplot as plt   
import time           
import numpy as np  
import os
import math
import random
from numpy import savetxt

parent = "/content/drive/MyDrive/CV project 2"

#parent = "Image Data"

#reading all the positive training images
path = parent + "/Training images (Pos)/"
Train_pos = [path+i for i in os.listdir(path)]

path = parent + "/Training images (Neg)/"
Train_neg = [path+i for i in os.listdir(path)]

path = parent + "/Test images (Pos)/"
Test_pos =[path+i for i in os.listdir(path)]

path= parent + "/Test images (Neg)/"
Test_neg = [path+i for i in os.listdir(path)]

print(Train_neg)
print(Train_pos)
print(Test_pos)
print(Test_neg)

#Converting all the images to gray_scale
def convert_to_gray(img): 
  #print(img)
  prop = img.shape
  
  #print(img[0][0])
  gray_img = np.zeros([img.shape[0],img.shape[1]])
  for i in range(prop[0]):
      for j in range( prop[1]):
          gray_img[i][j] = np.round(0.114*img[i][j][2] + 0.587*img[i][j][1] + 0.299*img[i][j][0])
  return gray_img

['/content/drive/MyDrive/CV project 2/Training images (Neg)/01-03e_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/00000057a_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/00000091a_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/no_person__no_bike_219_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/no_person__no_bike_259_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/00000053a_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/00000062a_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/00000093a_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/no_person__no_bike_213_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/no_person__no_bike_247_cut.bmp', '/content/drive/MyDrive/CV project 2/Training images (Neg)/01-03e_cut.txt', '/content/drive/MyDrive/CV project 2/Training images (Neg)/00000057a_cut.txt', '/con

In [ ]:
#Applying Prewitt's operator
def prewitt_operator(gray_img):
  h, w = np.shape(gray_img)[0], np.shape(gray_img)[1]
  # define filters
  horizontal = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])  # s2
  vertical = np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]])  # s1
  # define images with 0s
  newgradientImage = np.zeros((h, w))
  gradient_angles = np.zeros((h, w))
  horizontalGrad = np.zeros((h, w))
  verticalGrad = np.zeros((h, w))

  # offset by 1
  for i in range(1, h - 1):
      for j in range(1, w - 1):
        horizontalGrad[i][j] = np.sum(np.multiply(gray_img[i-1:i+2,j-1:j+2],horizontal))
        #print(gray_img[i-1:i+2,j-1:j+2])
        verticalGrad[i][j] = np.sum(np.multiply(gray_img[i-1:i+2,j-1:j+2],vertical))
          #print(verticalGrad,horizontalGrad)
        #print(horizontalGrad)
        #print(verticalGrad)
        mag = (np.sqrt(pow(horizontalGrad[i][j], 2.0) + pow(verticalGrad[i][j], 2.0)))
        #print(mag)
        #mag = abs(horizontalGrad[i][j])+abs(verticalGrad[i][j])
        if(horizontalGrad[i][j]==0 and verticalGrad[i][j]==0):
          mag=0
          angle = 0
        else:
          angle = math.degrees(math.atan(verticalGrad[i][j]/ horizontalGrad[i][j]))

        #converting negative angles to positive
        angle = angle%360 
        
        #converting them to range [0,180]
        if angle>=180:
          angle-=180
        #print("mag is",mag)
        gradient_angles[i,j] = angle
        newgradientImage[i,j] = mag
  #print(newgradientImage.shape)
  #print(horizontalGrad)
  #print(verticalGrad)
  # normalized_img1 = (255*(newgradientImage - np.min(newgradientImage))/np.ptp(newgradientImage)).astype(int)  
  normalized_img = (newgradientImage/np.amax(newgradientImage))*255
  normalized_img = normalized_img.astype(int)
  #normalized_img = (newgradientImage * np.max(newgradientImage))/255
  #print(normalized_img)
  #print(gradient_angles)

  return normalized_img,gradient_angles

In [ ]:
def calculate_histogram(array, weights):
  hist = np.zeros(9)
  weights = np.asarray(weights,dtype=float)
  array = np.asarray(array,dtype=float)

  #print(weights)
  centers = [10.0,30.0,50.0,70.0,90.0,110.0,130.0,150.0,170.0]
  bin1 = 0
  bin2 = 0
  proportion1=0.0
  proportion2 = 0.0
  for i in range(8):
      for j in range(8):
        if(array[i][j] in centers):
          proportion1 = 1.0
          proportion2 = 0.0
          bin1 = int(array[i][j]//20)

        elif(array[i][j]%20==0): #test corners
          proportion1 = 0.5
          proportion2 = 0.5
          if(array[i][j]==180.0 or array[i][j]==0.0):
            bin1 = 0
            bin2 = 8
          else:
            bin1 = math.ceil((array[i][j]/20)+0.05) - 1
            #hist[bin1] += 0.5*weights[i][j]
            #hist[bin1-1] += 0.5*weights[i][j]
            bin2 = bin1-1
        else:
          bin1 = math.ceil((array[i][j]/20)) - 1
          if(bin1==9):
            bin1 = 0
            bin2 = 8
          else:
            if(bin1==8 and array[i][j]>centers[bin1]):
              bin2 = 0
            elif(array[i][j]<centers[bin1] and bin1==0):
              bin2 = 8
            elif(array[i][j]<centers[bin1]):
              bin2 = bin1-1
            else:
              bin2 = bin1+1
          proportion1 = 1 - (abs(array[i][j]-centers[bin1])/20.0)

          proportion2 = 1-proportion1

        hist[bin1] = hist[bin1] + (proportion1*weights[i][j])
        hist[bin2] = hist[bin2] + (proportion2*weights[i][j])
  #hist,_ = np.histogram(array,bins=9,range=(0,180),weights=weights)
  #print(hist)
  return hist

In [ ]:
cell = [8, 8]
incr = [8,8]
bin_num = 9
im_size = [160,96]

def create_hog_features(grad_array,mag_array):
  max_h = int(((grad_array.shape[0]-cell[0])/incr[0])+1)
  max_w = int(((grad_array.shape[1]-cell[1])/incr[1])+1)
  cell_array = []
  w = 0
  h = 0
  i = 0
  j = 0
  #Creating 8X8 cells
  while i<max_h:
    w = 0
    j = 0

    while j<max_w:
      gradient_array = grad_array[h:h+cell[0],w:w+cell[1]]
      magnitude_array = mag_array[h:h+cell[0],w:w+cell[1]]
      val = calculate_histogram(gradient_array,magnitude_array)
      cell_array.append(val)
      j += 1
      w += incr[1]
    i += 1
    h += incr[0]

  cell_array = np.reshape(cell_array,(max_h, max_w, bin_num))
	#normalising blocks of cells
  block = [2,2]
	#here increment is 1

  max_h = int((max_h-block[0])+1)
  max_w = int((max_w-block[1])+1)
  #max_h = 19
  #max_w = 11
  block_list = []
  w = 0
  h = 0
  i = 0
  j = 0
  
  while i<max_h:
    w = 0
    j = 0
    while j<max_w:
      for_norm = cell_array[h:h+block[0],w:w+block[1]]
      #print(for_norm.shape)
      mag = np.linalg.norm(for_norm)
      arr_list = (for_norm/mag).flatten().tolist()
      #print("hello")
      #print(len(arr_list))
      block_list.extend(arr_list) #arr_list
      j += 1
      w += 1

    i += 1
    h += 1

	#returns a vextor array list of 7524 elements
  return block_list


images_hog_features = []



In [ ]:
def train():
  #Converting all the training images to gray and storing in an array
  gray_images = []
  for img in Train_pos:
    if img.endswith("bmp"):
      #print(img)
      image = cv2.imread(img)
      gray_img = convert_to_gray(image)
      gray_images.append((gray_img,1,img))

  for img in Train_neg:
    if img.endswith("bmp"):
      #print(img)
      image = cv2.imread(img)
      gray_img = convert_to_gray(image)
      gray_images.append((gray_img,0,img))
  #Applying prewitt operator and HOG to get the feature vectors of all the training images
  images_hog_features = []
  for i in gray_images:
    mags,angles = prewitt_operator(i[0])
    hog_features = create_hog_features(angles,mags)
    hog_features = np.asarray(hog_features,dtype=float)
    #hog_features = np.expand_dims(hog_features,axis=0)
    hog_features = np.nan_to_num(hog_features)
    #print(hog_features.shape)
    savetxt(i[2].replace(".bmp",".txt"),hog_features.flatten())
    images_hog_features.append(hog_features)
  label = [i[1] for i in gray_images]
  return images_hog_features,label

In [ ]:
def test(images_hog_features,label):
  for img in Test_neg + Test_pos:
    test_img = cv2.imread(img)
    test_gray = convert_to_gray(test_img)
    magnitude,angle = prewitt_operator(test_gray)
    hog_features = create_hog_features(angle,magnitude)
    hog_features = np.asarray(hog_features,dtype=float)
    #test_hog_features = np.expand_dims(hog_features,axis=0)
    print(hog_features.shape)
    hog_features = np.nan_to_num(hog_features)
    distances = []
    
    images_hog_features = np.array(images_hog_features)
    print(images_hog_features.shape)
    #new_images_hog_features = images_hog_features[0]

    for i in range(len(images_hog_features)):
      sum = 0
      sum_min = 0
      for j in range(7524):
        sum+=images_hog_features[i][j]
        sum_min += min(images_hog_features[i][j],hog_features[j])
      distances.append(sum_min/sum)

    dictionary  = dict(zip(distances,label))
    dict_new = [(i,j) for i,j in dictionary.items()]
    dict_new.sort(key=lambda tup: tup[0],reverse=True)

    neighbors = list()
    for i in range(3):
      neighbors.append(dict_new[i][1])
    print(neighbors)
    prediction = max(neighbors, key = neighbors.count)
    if "Neg" in img:
      print("No-Human")
    else:
      print("Human")
    print("Human" if prediction else "No-Human")
    print("\n")

In [ ]:
images_hog_features,label = train()
test(images_hog_features,label)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: RuntimeWarning: invalid value encountered in true_divide


(7524,)
(20, 7524)
[1, 0, 0]
No-Human
No-Human


(7524,)
(20, 7524)
[0, 0, 1]
No-Human
No-Human


(7524,)
(20, 7524)
[0, 0, 0]
No-Human
No-Human


(7524,)
(20, 7524)
[0, 1, 1]
No-Human
Human


(7524,)
(20, 7524)
[1, 0, 0]
No-Human
No-Human


(7524,)
(20, 7524)
[1, 0, 0]
Human
No-Human


(7524,)
(20, 7524)
[0, 1, 1]
Human
Human


(7524,)
(20, 7524)
[1, 1, 1]
Human
Human


(7524,)
(20, 7524)
[0, 1, 1]
Human
Human


(7524,)
(20, 7524)
[1, 1, 1]
Human
Human




In [ ]:
np.round(3.5)

4.0